In [4]:
import os
os.environ['QT_API'] = 'pyqt'
os.environ['ETS_TOOLKIT'] = 'qt4'
#os.environ['SUBJECTS_DIR'] = "/path/to/subjects"

import sip
sip.setapi('QDate',2)

import os
import os.path as op
import numpy as np
from numpy import arange
from numpy.random import permutation
import nibabel as nib
from surfer import Brain, utils
import pandas as pd
print(__doc__)


%gui qt


Directories and names



In [114]:
"""
Directories and names
"""
subject_id = "fsaverage"
subjects_dir = os.environ["SUBJECTS_DIR"]

#csv files containing target coordinates
lh_file = '/Users/thackery/Dropbox/scripts/plotting/PySurfer_plottingscripts/lh_coords2.csv'
rh_file = '/Users/thackery/Dropbox/scripts/plotting/PySurfer_plottingscripts/rh_coords2.csv'



In [115]:
"""
Flags and UI
"""

#Set number of datasets (studies, conditions, any other grouping for individual coordinate spheres)
ndatasets = 4#9#

#set a scaling factor for spheres. Sizes from 0.25 - 0.5 yield decent results
blobscale = 0.35

#flag for using custom (relative) blob sizes per sphere (e.g., from different recorded cluster extents)
resizeblobs = 0

In [127]:
"""
read in coordinate data
"""
#data should be structured with 5 columns - x y z grp scale
#grp is a grouping factor you choose - same study, same cognitive process, whatever
#scale is a scaling factor used when "resizeblobs=1" - could be t-value magnitude, cluster size (k), whatever
lh_dat = pd.read_csv(lh_file)
rh_dat = pd.read_csv(rh_file)

In [117]:
"""
visualize dataframe
"""
rh_dat.head()

,x,y,z,grp,scale
0,26,57,-8,1,45
1,28,38,31,1,45
2,28,55,-1,1,45
3,30,57,-5,0,45
4,36,63,6,0,5


In [133]:
lh_dat.head()

,x,y,z,grp,scale
0,-10,66,8,0,5
1,18,57,0,0,5
2,-18,58,-8,3,10
3,-24,62,22,1,19
4,-9,59,28,0,383


In [128]:
'''
convert coords to numpy lists
'''

rh_dat_coords = rh_dat[['x', 'y', 'z']]
#rh_dat_coords.head
coords_rh = rh_dat_coords.values.tolist()
print(coords_rh)

#coords_rh_studkey = rh_dat.grp.tolist()
#coords_rh_sz_key = rh_dat.scale.tolist()

lh_dat_coords = lh_dat[['x', 'y', 'z']]
coords_lh = lh_dat_coords.values.tolist()
print(coords_lh)

[[26, 57, -8], [28, 38, 31], [28, 55, -1], [30, 57, -5], [36, 63, 6], [27, 62, 22], [9, 59, 34], [26, 64, 10], [30, 52, 8], [30, 48, 8], [20, 68, 10], [32, 56, 0], [36, 54, 18], [38, 62, 8]]
[[-10, 66, 8], [18, 57, 0], [-18, 58, -8], [-24, 62, 22], [-9, 59, 28], [-21, 56, -8], [-34, 52, -4], [-30, 62, 18], [-40, 52, 8]]


In [119]:
"""
Create generator of unique RGB colors for plotting objects (such as coordinate spheres)
"""

import math

v = 1.0
s = 1.0
p = 0.0
def rgbcolor(h, f):
    """Convert a color specified by h-value and f-value to an RGB
    three-tuple."""
    # q = 1 - f
    # t = f
    if h == 0:
        return v, f, p
    elif h == 1:
        return 1 - f, v, p
    elif h == 2:
        return p, v, f
    elif h == 3:
        return p, 1 - f, v
    elif h == 4:
        return f, p, v
    elif h == 5:
        return v, p, 1 - f

def uniquecolors(n):
    """Compute a list of distinct colors, ecah of which is
    represented as an RGB three-tuple"""
    hues = [360.0 / n * i for i in range(n)]
    hs = (math.floor(hue / 60) % 6 for hue in hues)
    fs = (hue / 60 - math.floor(hue / 60) for hue in hues)
    return [rgbcolor(h, f) for h, f in zip(hs, fs)]

In [120]:
"""
Generate sphere colors
"""
datacolors=uniquecolors(ndatasets)
datacolors

[(1.0, 0.0, 0.0), (0.5, 1.0, 0.0), (0.0, 1.0, 1.0), (0.5, 0.0, 1.0)]

In [121]:
"""
First we'll get a set of stereotaxic foci in the MNI
coordinate system. These might be peak activations from
a volume based analysis.
"""
#coords_rh_studkey = [1,1,1,1,2,4,4,5,6,6,7,7,8,8] #study
#coords_rh_studkey = [1,1,1,0,0,1,0,0,1,1,2,2,1,1] #cognition
#scaling factor (in this example, multiples of recorded cluster sizes relative to smallest size)
#coords_rh_sz_key = [9,9,9,9,1,3.8,76.6,5.8,28,28,31.2,31.2,21.8,21.8]#ratio against smallest k
#coords_rh_sz_key = [45,45,45,45,5,19,383,29,140,140,156,156,109,109]#raw k


# coords_lh = [[-10, 66, 8],
#          [-18,57,0],
#          [-18,58,-8],
#          [-24,62,22],
#          [-9,59,28],
#          [-21,56,-8],
#          [-34,52,-4],
#              [-30,62,18],
#              [-40,52,8]
#                       ]

# #coords_lh_studkey = [0,2,3,4,4,4,5,8,8] #study
# coords_lh_studkey = [0,0,3,1,0,0,0,1,1] #cognition

# #scaling factor (in this example, multiples of recorded cluster sizes relative to smallest size)
# #coords_lh_sz_key = [1,1,2,3.8,76.6,3.8,30.2,21.8,21.8]#ratio against smallest k
# coords_lh_sz_key = [5,5,10,19,383,19,151,109,109]#raw k



# print(coords_rh_studkey)

"\nFirst we'll get a set of stereotaxic foci in the MNI\ncoordinate system. These might be peak activations from\na volume based analysis.\n"

In [122]:
"""

Add Yeo reference labels
"""

hemi = "lh"

"""
Read in the Buckner resting state network annotation. (This requires a
relatively recent version of Freesurfer, or it can be downloaded separately).
"""
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".Yeo2011_17Networks_N1000.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)


"""
Make a random vector of scalar data corresponding to a value for each region in
the parcellation.

"""
rs = np.random.RandomState(4)
roi_data = rs.uniform(.5, .75, size=len(names))

"""
Make a vector containing the data point at each vertex.
"""
vtx_data_lh = roi_data[labels]


"""
repeat for rh
"""
hemi = "rh"
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi + ".Yeo2011_17Networks_N1000.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

vtx_data_rh = roi_data[labels]

## Deal with Left Hemisphere Data

In [129]:
currhem = "lh"

"""
Bring up the visualization.
"""
brain_lh = Brain(subject_id, currhem, "inflated")

#brain = Brain("fsaverage", "split", "inflated",
#              views=['frontal'], background="white")

In [130]:
"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.

https://matplotlib.org/examples/color/colormaps_reference.html
"""
#brain.add_data(vtx_data_lh, .5, .75, colormap="GnBu", alpha=.9, hemi="lh")
#brain.add_data(vtx_data_rh, .5, .75, colormap="GnBu", alpha=.9, hemi="rh")

brain_lh.add_data(vtx_data_lh, .5, .75, colormap="RdYlBu", alpha=.8, hemi="lh")
#brain_lh.close()


In [131]:
"""
Map foci to this hemisphere.

Now we plot the foci on the inflated surface. We will map
the foci onto the surface by finding the vertex on the "white"
mesh that is closest to the coordinate of each point we want
to display.

While this is not a perfect transformation, it can give you
some idea of where peaks from a volume-based analysis would
be located on the surface.

You can use any valid matplotlib color for the foci; the
default is white.



This works through iterating through a list of datapoints, such that data from the same grouping gets \\
the same color code
"""
#for i in range(4,6):
#    subcoords = eval("coords_lh_s" + str(i))
#    print(subcoords)
#    brain_lh.add_foci(subcoords, map_surface="white", color=datacolors[i], scale_factor = 0.5, hemi="lh")    


#for i in coords_lh:
for i in range(len(coords_lh)):
    if resizeblobs == 1:
     brain_lh.add_foci(coords_lh[i], map_surface="white", color=datacolors[lh_dat.grp[i]], scale_factor = blobscale+(0.01*lh_dat.scale[i]), hemi="lh") #in this scheme, convert raw k into a percentage, and add to a base sphere size   
    else:
     brain_lh.add_foci(coords_lh[i], map_surface="white", color=datacolors[lh_dat.grp[i]], scale_factor = blobscale, hemi="lh")   




#brain_lh.add_foci(coords_lh_s1, map_surface="white", color="b", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s3, map_surface="white", color="r", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s4, map_surface="white", color="c", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s5, map_surface="white", color="m", scale_factor = 0.5, hemi="lh")
#brain_lh.add_foci(coords_lh_s6, map_surface="white", color="y", scale_factor = 0.5, hemi="lh")


[-10, 66, 8]
0
[18, 57, 0]
0
[-18, 58, -8]
3
[-24, 62, 22]
1
[-9, 59, 28]
0
[-21, 56, -8]
0
[-34, 52, -4]
0
[-30, 62, 18]
1
[-40, 52, 8]
1


In [132]:
"""
Various view and saving options do not play well with 'split' brain, and are best dealt with on a hemisphere basis
"""

#brain_rh.show_view(dict(azimuth=-42, elevation=105, distance=225,
#                     focalpoint=[-30, -20, 15]))

brain_lh.show_view('frontal')

((119.99999999999999,
  80.0,
  434.03852844238281,
  array([ 0.        ,  0.48182297,  0.        ])),
 106.739)

In [357]:
"""
save output
"""
#with multiple arguments, one can use a tuple as operand: 'foo %d, bar %d' % (foo, bar)
brain_lh.save_image("%s_lat_%s.png" % (subject_id, currhem))

## Deal with Right Hemisphere Data

In [109]:
currhem = "rh"

"""
Bring up the visualization.
"""
brain_rh = Brain(subject_id, currhem, "inflated")

In [110]:
"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
brain_rh.add_data(vtx_data_rh, .5, .75, colormap="RdYlBu", alpha=.8, hemi="rh")


In [111]:
#print(len(coords_rh))
#print(len(coords_rh_studkey))
#print(coords_rh[11])
#print(range(len(coords_rh)))

for i in range(len(coords_rh)):
    if resizeblobs == 1:
     #rain_rh.add_foci(coords_rh[i], map_surface="white", color=datacolors[rh_dat.grp[i]], scale_factor = blobscale+(0.01*coords_rh_sz_key[i]), hemi="rh")    
     brain_rh.add_foci(coords_rh[i], map_surface="white", color=datacolors[rh_dat.grp[i]], scale_factor = blobscale+(0.01*rh_dat.scale[i]), hemi="rh")    
    else:
     #brain_rh.add_foci(coords_rh[i], map_surface="white", color=datacolors[coords_rh_studkey[i]], scale_factor = blobscale, hemi="rh")   
     brain_rh.add_foci(coords_rh[i], map_surface="white", color=datacolors[rh_dat.grp[i]], scale_factor = blobscale, hemi="rh")   


In [346]:
brain_rh.show_view('frontal')

((59.999999999999993,
  80.0,
  448.60737609863281,
  array([-1.48587799,  4.42029953,  0.        ])),
 -106.739)

In [347]:
"""
save output
"""
#with multiple arguments, one can use a tuple as operand: 'foo %d, bar %d' % (foo, bar)
brain_rh.save_image("%s_lat_%s.png" % (subject_id, currhem))

In [348]:
###############################################################################
# # Get a set of images as a montage, note the data could be saved if desired
# #image = brain_rh.save_montage(None, ['l', 'v', 'm'], orientation='v')
# image = brain.save_montage(None, ['l', 'v', 'm'], orientation='v')
# brain.close()



In [349]:
###############################################################################
# View created image
# import pylab as pl
# fig = pl.figure(figsize=(5, 3), facecolor='white')
# ax = pl.axes(frameon=False)
# ax.imshow(image, origin='upper')
# pl.xticks(())
# pl.yticks(())
# pl.draw()
# pl.show()

In [350]:
#utils.coord_to_label(subject_id, coords_rh[4], label='coord', #example_data/
#                     hemi='rh', n_steps=25, map_surface="white")
#brain_rh.add_label('coord-rh.label', color="darkseagreen", alpha=.8)